In [1]:
import pandas as pd

Loading data

In [2]:
customers = pd.read_csv('../data/Customers.csv')
products = pd.read_csv('../data/Products.csv')
transactions = pd.read_csv('../data/Transactions.csv')

Merge the dataset into one file with common IDs


In [3]:
merged_data = transactions.merge(customers, on="CustomerID").merge(products, on="ProductID")

In [4]:
merged_data.head()

,TransactionID,CustomerID,ProductID,TransactionDate,Quantity,TotalValue,Price_x,CustomerName,Region,SignupDate,ProductName,Category,Price_y
0,T00001,C0199,P067,2024-08-25 12:38:23,1,300.68,300.68,Andrea Jenkins,Europe,2022-12-03,ComfortLiving Bluetooth Speaker,Electronics,300.68
1,T00112,C0146,P067,2024-05-27 22:23:54,1,300.68,300.68,Brittany Harvey,Asia,2024-09-04,ComfortLiving Bluetooth Speaker,Electronics,300.68
2,T00166,C0127,P067,2024-04-25 07:38:55,1,300.68,300.68,Kathryn Stevens,Europe,2024-04-04,ComfortLiving Bluetooth Speaker,Electronics,300.68
3,T00272,C0087,P067,2024-03-26 22:55:37,2,601.36,300.68,Travis Campbell,South America,2024-04-11,ComfortLiving Bluetooth Speaker,Electronics,300.68
4,T00363,C0070,P067,2024-03-21 15:10:10,3,902.04,300.68,Timothy Perez,Europe,2022-03-15,ComfortLiving Bluetooth Speaker,Electronics,300.68


In [5]:
merged_data.isnull().sum()

TransactionID      0
CustomerID         0
ProductID          0
TransactionDate    0
Quantity           0
TotalValue         0
Price_x            0
CustomerName       0
Region             0
SignupDate         0
ProductName        0
Category           0
Price_y            0
dtype: int64

In [6]:
merged_data.columns

Index(['TransactionID', 'CustomerID', 'ProductID', 'TransactionDate',
       'Quantity', 'TotalValue', 'Price_x', 'CustomerName', 'Region',
       'SignupDate', 'ProductName', 'Category', 'Price_y'],
      dtype='object')

In [7]:
merged_data.describe()

,Quantity,TotalValue,Price_x,Price_y
count,1000.000000,1000.000000,1000.00000,1000.00000
mean,2.537000,689.995560,272.55407,272.55407
std,1.117981,493.144478,140.73639,140.73639
min,1.000000,16.080000,16.08000,16.08000
25%,2.000000,295.295000,147.95000,147.95000
50%,3.000000,588.880000,299.93000,299.93000
75%,4.000000,1011.660000,404.40000,404.40000
max,4.000000,1991.040000,497.76000,497.76000


In [8]:
merged_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   TransactionID    1000 non-null   object 
 1   CustomerID       1000 non-null   object 
 2   ProductID        1000 non-null   object 
 3   TransactionDate  1000 non-null   object 
 4   Quantity         1000 non-null   int64  
 5   TotalValue       1000 non-null   float64
 6   Price_x          1000 non-null   float64
 7   CustomerName     1000 non-null   object 
 8   Region           1000 non-null   object 
 9   SignupDate       1000 non-null   object 
 10  ProductName      1000 non-null   object 
 11  Category         1000 non-null   object 
 12  Price_y          1000 non-null   float64
dtypes: float64(3), int64(1), object(9)
memory usage: 101.7+ KB


The goal is to create a single row per customer by summarizing their transaction history and product preferences. Since each customer has multiple transactions, we use aggregation functions to extract meaningful information. 

In [9]:
customer_features = merged_data.groupby("CustomerID").agg({
    "TotalValue": "sum",
    "Quantity": "sum",
    "Price_x": "mean",
    "Region": lambda x: x.mode()[0], 
    "Category": lambda x: x.mode()[0]
}).reset_index()

In [10]:
customer_features.head()

,CustomerID,TotalValue,Quantity,Price_x,Region,Category
0,C0001,3354.52,12,278.334000,South America,Electronics
1,C0002,1862.74,10,208.920000,Asia,Clothing
2,C0003,2725.38,14,195.707500,South America,Home Decor
3,C0004,5354.88,23,240.636250,South America,Books
4,C0005,2034.24,7,291.603333,Asia,Electronics


One Hot Encoding

In [11]:
customer_features = pd.get_dummies(customer_features, columns=["Region", "Category"])
customer_features.head()

,CustomerID,TotalValue,Quantity,Price_x,Region_Asia,Region_Europe,Region_North America,Region_South America,Category_Books,Category_Clothing,Category_Electronics,Category_Home Decor
0,C0001,3354.52,12,278.334000,False,False,False,True,False,False,True,False
1,C0002,1862.74,10,208.920000,True,False,False,False,False,True,False,False
2,C0003,2725.38,14,195.707500,False,False,False,True,False,False,False,True
3,C0004,5354.88,23,240.636250,False,False,False,True,True,False,False,False
4,C0005,2034.24,7,291.603333,True,False,False,False,False,False,True,False


In [12]:
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import cosine_similarity

Z-score normalization

In [13]:
scaler = StandardScaler()
feature_matrix = scaler.fit_transform(customer_features.drop(columns=["CustomerID"]))
feature_matrix

array([[-0.06170143, -0.12203296,  0.09467022, ..., -0.54056248,
         1.84992492, -0.51721942],
       [-0.87774353, -0.44800021, -0.90401592, ...,  1.84992492,
        -0.54056248, -0.51721942],
       [-0.40585722,  0.20393428, -1.09410928, ..., -0.54056248,
        -0.54056248,  1.93341543],
       ...,
       [-1.38697529, -1.58888557, -0.46110018, ...,  1.84992492,
        -0.54056248, -0.51721942],
       [-0.81399315, -0.61098383, -0.30420572, ..., -0.54056248,
         1.84992492, -0.51721942],
       [ 0.70636652,  0.52990153,  0.35611784, ...,  1.84992492,
        -0.54056248, -0.51721942]], shape=(199, 11))

Cosine Similarity

In [14]:
# Compute Similarity
similarity_matrix = cosine_similarity(feature_matrix)
customer_ids = customer_features["CustomerID"].values

In [15]:
similarity_matrix

array([[ 1.        , -0.27858075,  0.23062831, ..., -0.2373934 ,
         0.3660242 , -0.30170922],
       [-0.27858075,  1.        , -0.12880515, ...,  0.46534525,
        -0.12139046,  0.75812147],
       [ 0.23062831, -0.12880515,  1.        , ..., -0.16736918,
        -0.2171534 , -0.32129318],
       ...,
       [-0.2373934 ,  0.46534525, -0.16736918, ...,  1.        ,
         0.44524922,  0.10707049],
       [ 0.3660242 , -0.12139046, -0.2171534 , ...,  0.44524922,
         1.        , -0.35857678],
       [-0.30170922,  0.75812147, -0.32129318, ...,  0.10707049,
        -0.35857678,  1.        ]], shape=(199, 199))

PCA

In [16]:
from sklearn.decomposition import PCA

In [17]:
pca = PCA(n_components=10)  # Choose based on explained variance
pca_features = pca.fit_transform(feature_matrix)

In [18]:
pca_features

array([[ 4.61952146e-01, -1.11814418e+00,  2.02985786e+00, ...,
        -2.65432871e-01, -1.82581907e-02, -1.94573551e-15],
       [-1.57987179e+00,  1.76001998e+00,  9.02996885e-01, ...,
        -3.66195509e-01, -5.81675940e-02, -1.30735727e-15],
       [-3.71673061e-01, -1.87114628e+00,  5.79096084e-01, ...,
        -1.66119618e-01, -1.90394099e-01, -2.01595883e-15],
       ...,
       [-2.72112663e+00,  2.91272023e-01, -5.68088574e-01, ...,
         2.88396598e-01,  1.45244966e-01,  2.13433411e-15],
       [-1.13787391e+00, -9.61256358e-01,  5.98958172e-01, ...,
        -9.65003275e-01, -1.32094894e-01,  2.31127590e-15],
       [ 3.95487375e-01,  2.04756850e+00,  8.93714138e-01, ...,
         3.96952578e-01,  8.60492570e-02, -8.63268061e-16]],
      shape=(199, 10))

In [19]:
similarity_matrix_pca = cosine_similarity(pca_features)

In [20]:
similarity_matrix_pca

array([[ 1.        , -0.27858075,  0.23062831, ..., -0.2373934 ,
         0.3660242 , -0.30170922],
       [-0.27858075,  1.        , -0.12880515, ...,  0.46534525,
        -0.12139046,  0.75812147],
       [ 0.23062831, -0.12880515,  1.        , ..., -0.16736918,
        -0.2171534 , -0.32129318],
       ...,
       [-0.2373934 ,  0.46534525, -0.16736918, ...,  1.        ,
         0.44524922,  0.10707049],
       [ 0.3660242 , -0.12139046, -0.2171534 , ...,  0.44524922,
         1.        , -0.35857678],
       [-0.30170922,  0.75812147, -0.32129318, ...,  0.10707049,
        -0.35857678,  1.        ]], shape=(199, 199))

CSV File

In [21]:
lookalike_dict = {}
for i in range(20):
    cust_id = customer_ids[i]
    sim_scores = list(enumerate(similarity_matrix[i]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)[1:4]  # Exclude self
    lookalike_dict[cust_id] = [(customer_ids[idx], round(score, 2)) for idx, score in sim_scores]

In [22]:
lookalike_df = pd.DataFrame(lookalike_dict.items(), columns=["CustomerID", "Lookalikes"])
lookalike_df.to_csv("Lookalike.csv", index=False)

In [23]:
lookalike_df.head()

,CustomerID,Lookalikes
0,C0001,"[(C0181, 0.98), (C0120, 0.97), (C0184, 0.96)]"
1,C0002,"[(C0088, 0.99), (C0106, 0.96), (C0134, 0.93)]"
2,C0003,"[(C0031, 0.95), (C0052, 0.95), (C0195, 0.94)]"
3,C0004,"[(C0165, 0.96), (C0169, 0.96), (C0087, 0.95)]"
4,C0005,"[(C0140, 1.0), (C0186, 0.99), (C0146, 0.98)]"


CSV after PCA

In [24]:
lookalike_dict_pca = {}
for i in range(20):
    cust_id = customer_ids[i]
    sim_scores = list(enumerate(similarity_matrix_pca[i]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)[1:4]  # Exclude self
    lookalike_dict_pca[cust_id] = [(customer_ids[idx], round(score, 2)) for idx, score in sim_scores]

In [25]:
lookalike_df_pca = pd.DataFrame(lookalike_dict.items(), columns=["CustomerID", "Lookalikes_PCA"])
lookalike_df_pca.to_csv("Lookalike_PCA.csv", index=False)

In [26]:
lookalike_df_pca.head()

,CustomerID,Lookalikes_PCA
0,C0001,"[(C0181, 0.98), (C0120, 0.97), (C0184, 0.96)]"
1,C0002,"[(C0088, 0.99), (C0106, 0.96), (C0134, 0.93)]"
2,C0003,"[(C0031, 0.95), (C0052, 0.95), (C0195, 0.94)]"
3,C0004,"[(C0165, 0.96), (C0169, 0.96), (C0087, 0.95)]"
4,C0005,"[(C0140, 1.0), (C0186, 0.99), (C0146, 0.98)]"


We selected features like total value, quantity, and category to capture customer behavior and preferences. PCA was applied for dimensionality reduction, improving model efficiency and removing noise. Cosine similarity was used to measure similarity, and recommendations were saved for analysis.